# Méthodes de projection

In [1]:
using LinearAlgebra

## Gradient projeté

### Algorithme de recherche linéaire de base

L'approche la plus simple consiste à exploiter le fait que $\forall\, t > 0$,
$$
d = P_{\mathcal{X}}(x - t\nabla f(x)) - x
$$
est une direction de descente. Si nous fixons la valeur de $t$, par exemple $t = 1$, nous pouvons directement adapter notre méthode de recherche linéaire, en utilisant la méthode de marche arrière d'Armijo.

In [2]:
function linesearch(f::Function, g::Function, h:: Function,
        x0::Vector, direction::Function, steplength::Function,
        δ::Float64 = 1e-6, stop::Function = stopping, nmax::Int64 = 1000)

    k = 0
    x = x0
    δ2 = δ*δ
    n = length(x)

    # Compute the search direction
    d, dfx = direction(f,g,h,x)

    while (stop(d, k, nmax, δ2) == false)
        # Compute the step length along d
        α = steplength(f,dfx,x,d)
        # Update the iterate
        x += α*d
        # Compute the search direction
        d, dfx = direction(f,g,h,x)
        k += 1
    end

    return x, dfx, k
end

linesearch (generic function with 4 methods)

In [3]:
function ArmijoStep(f::Function, dfx::Vector, x::Vector, d:: Vector,
    αmax:: Float64 = 1.0, β:: Float64 =0.1, κ:: Float64 =0.2)
    
    α = αmax
    
    fx = f(x)
    fxcand = f(x+α*d)

    while ((fxcand >= fx+α*β*dot(dfx,d)) && (α > 1e-16))
        α *= κ
        fxcand = f(x+α*d)        
    end
    
    return α
end

ArmijoStep (generic function with 4 methods)

Nous allons considérer des contraintes de bornes. La projection est alors très facile à obtenir.

In [4]:
function boundsProjection(x::Vector, lower::Vector, upper::Vector)
    # we assume upper .>= lower
    return max.(min.(upper,x),lower)
end

boundsProjection (generic function with 1 method)

Testons la dernière fonction.

In [5]:
l = [ 0 for i = 1:10 ]
u = [ 10 for i = 1:10 ]
x = [ -5+2.0*i for i = 1:10 ]
z = boundsProjection(x,l,u)

10-element Array{Float64,1}:
  0.0
  0.0
  1.0
  3.0
  5.0
  7.0
  9.0
 10.0
 10.0
 10.0

### Exemple

Nous allons considérer la fonction d'exemple donnée dans le chapitre d'introduction de "Trust-Region Methods" (Conn, Gould, and Toint, 2000):
$$
f(x,y) = -10x^2+10y^2+4\sin(xy)-2x+x^4
$$

In [6]:
using ForwardDiff

f(x::Vector) = -10*x[1]^2+10*x[2]^2+4*sin(x[1]*x[2])-2*x[1]+x[1]^4

f (generic function with 1 method)

In [7]:
using Plots
plotly()

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots C:\Users\slash\.julia\packages\Plots\jZRxE\src\backends.jl:373


Plots.PlotlyBackend()

In [33]:
default(size=(600,600), fc=:heat)
x, y = -3.0:0.1:3.0, -3.0:0.1:3.0
z = Surface((x,y)->f([x,y]), x, y)
surface(x,y,z)

<!DOCTYPE html>
 
 
 Plots.jl

In [21]:
Plots.contour(x,y,z, linealpha = 0.1, levels=2000)

<!DOCTYPE html>
 
 
 Plots.jl

In [10]:
g = x -> ForwardDiff.gradient(f, x);
H = x -> ForwardDiff.hessian(f, x)

#11 (generic function with 1 method)

In [11]:
function stopping_unconstrained(x::Vector, ∇x::Vector, k::Int, nmax::Int, tol::Float64)
    if (dot(∇x,∇x) < tol)
        return true;
    elseif (k >= nmax)
        return true;
    else
        return false;
    end
end

stopping_unconstrained (generic function with 1 method)

Nous pouvons définir certaines directions de recherche habituelles.

In [12]:
function NewtonDirection(f::Function, g:: Function, h:: Function, x::Vector)
    n = length(x)
    ∇f = g(x)
    H = h(x)
    return -H\∇f, ∇f
end

NewtonDirection (generic function with 1 method)

In [13]:
function steepestDirection(f::Function, g:: Function, h:: Function, x::Vector)
    n = length(x)
    ∇f = g(x)
    return -∇f, ∇f
end

steepestDirection (generic function with 1 method)

In [14]:
function stopping(d::Vector, k::Int, nmax::Int, tol::Float64)
    if (dot(d,d) < tol)
        return true;
    elseif (k >= nmax)
        return true;
    else
        return false;
    end
end

stopping (generic function with 1 method)

La minimisation sans contrainte donne

In [15]:
x, df, k = linesearch(f, g, H, [0.0,0.0], steepestDirection, ArmijoStep)

([2.3066301125619457, -0.3323086499407665], [-6.694542946661386e-7, -5.345261122613465e-8], 58)

In [16]:
f(x)

-31.180733385187978

#### Contraintes

Considérons les bornes simples:
$$
0 \leq x[i] \leq 10,\ \forall i
$$

In [22]:
l = [0.0; 0.0]
u = [10.0; 10.0]

2-element Array{Float64,1}:
 10.0
 10.0

In [23]:
mutable struct Bounds
    l::Array{Float64,1}
    u::Array{Float64,1}
end

function (projectBounds::Bounds)(x::Vector)
    return max.(min.(projectBounds.u,x),projectBounds.l)
end

In [24]:
projectBounds = Bounds(l, u)

Bounds([0.0, 0.0], [10.0, 10.0])

In [25]:
projectBounds.l = [0.0; 0.0]
projectBounds.u = [Inf; Inf]

2-element Array{Float64,1}:
 Inf
 Inf

In [27]:
function projectedDirection(f::Function, g:: Function, h:: Function, x::Vector)
    n = length(x)

    ∇f = g(x)

    d = x-∇f
    d = projectBounds(d)
    d = d-x

    return d, ∇f
end

projectedDirection (generic function with 1 method)

In [28]:
x, df, k = linesearch(f, g, H, [1.0,1.0], projectedDirection, ArmijoStep)

([2.284484124446568, 0.0], [-7.202996954447372e-7, 9.137936497786272], 43)

In [31]:
x, f(x)

([2.284484124446568, 0.0], -29.521065172290175)

## Chemin projeté

In [35]:
function (projectDirection::Bounds)(d::Vector, x::Vector)
    d += x
    d = max.(min.(projectDirection.u,d),projectDirection.l)
    d -= x

    return d
end

In [36]:
projectDirection = Bounds(l, u)

Bounds([0.0, 0.0], [10.0, 10.0])

In [37]:
function linesearchProjection(f::Function, g::Function, h:: Function,
        x0::Vector, direction::Function, step::Function,
        δ::Float64 = 1e-6, stop::Function = stopping, nmax::Int64 = 1000)

    k = 0
    x = x0
    δ2 = δ*δ
    n = length(x)

    # Compute the search direction
    d, ∇f = direction(f,g,h,x)
    s = projectDirection(d, x)

    while (stop(s, k, nmax, δ2) == false)
        # Compute the step length along d
        s = step(f,∇f,x,d)
        # Update the iterate
        x += s
        # Compute the search direction
        d, ∇f = direction(f,g,h,x)
        s = projectDirection(d, x)
        k += 1
    end

    return x, ∇f, k
end

linesearchProjection (generic function with 4 methods)

In [29]:
function GoldsteinStep(f::Function, ∇f::Vector, x::Vector, d:: Vector,
    αmax:: Float64 = 1.0, β:: Float64 =0.1, κ:: Float64 =0.2)

    α = αmax
    
    fx = f(x)
    # Project d
    s = α*d 
    s = projectDirection(s, x)
    fxcand = f(x+s)

    while ((fxcand >= fx+β*dot(∇f,s)) && (α > 1e-16))
        α *= κ 
        # Project d        
        s = α*d
        s = projectDirection(s, x)
        fxcand = f(x+s)
    end
    
    return s
end

GoldsteinStep (generic function with 4 methods)

In [40]:
x, dfx, k = linesearchProjection(f, g, H, [1.0,1.0], steepestDirection, GoldsteinStep)

([2.2844841247197536, 0.0], [-7.086547810786215e-7, 9.137936498879014], 42)

In [44]:
k, x, f(x)

(42, [2.2844841247197536, 0.0], -29.521065172290175)

Le choix de $\alpha$max influence influence le comportement de l'algorithme. Nous allons définir une structure permettant de modifier facilement les paramètres de la recherche linéaire.

In [41]:
abstract type Step end

mutable struct LineSearchStep <: Step 
    αmax::Float64
    β::Float64
    κ::Float64
end

goldsteinstep = LineSearchStep(1.0, 0.1, 0.2)

LineSearchStep(1.0, 0.1, 0.2)

In [42]:
function (goldsteinstep::LineSearchStep)(f::Function, ∇f::Vector, x::Vector, d:: Vector)

    α = goldsteinstep.αmax
    
    fx = f(x)
    # Project d
    s = α*d
    s = projectDirection(s, x)
    fxcand = f(x+s)

    while ((fxcand >= fx+goldsteinstep.β*dot(∇f,s)) && (α > 1e-16))
        α *= goldsteinstep.κ
        # Project d        
        s = α*d
        s = projectDirection(s, x)
        fxcand = f(x+s)
    end
    
    return s
end

In [44]:
function linesearchProjection(f::Function, g::Function, h:: Function,
        x0::Vector, direction::Function, step::Step,
        δ::Float64 = 1e-6, stop::Function = stopping, nmax::Int64 = 1000)

    k = 0
    x = x0
    δ2 = δ*δ
    n = length(x)

    # Compute the search direction
    d, ∇f = direction(f,g,h,x)
    s = projectDirection(d, x)

    while (stop(s, k, nmax, δ2) == false)
        # Compute the step length along d
        s = step(f,∇f,x,d)
        # Update the iterate
        x += s
        # Compute the search direction
        d, ∇f = direction(f,g,h,x)
        s = projectDirection(d, x)
        k += 1
    end

    return x, ∇f, k
end

linesearchProjection (generic function with 8 methods)

In [45]:
methods(linesearchProjection)

# 8 methods for generic function "linesearchProjection":
[1] linesearchProjection(f::Function, g::Function, h::Function, x0::Array{T,1} where T, direction::Function, step::Function) in Main at In[37]:1
[2] linesearchProjection(f::Function, g::Function, h::Function, x0::Array{T,1} where T, direction::Function, step::Function, δ::Float64) in Main at In[37]:1
[3] linesearchProjection(f::Function, g::Function, h::Function, x0::Array{T,1} where T, direction::Function, step::Function, δ::Float64, stop::Function) in Main at In[37]:1
[4] linesearchProjection(f::Function, g::Function, h::Function, x0::Array{T,1} where T, direction::Function, step::Function, δ::Float64, stop::Function, nmax::Int64) in Main at In[37]:1
[5] linesearchProjection(f::Function, g::Function, h::Function, x0::Array{T,1} where T, direction::Function, step::Step) in Main at In[44]:1
[6] linesearchProjection(f::Function, g::Function, h::Function, x0::Array{T,1} where T, direction::Function, step::Step, δ::Float64) in Main at In[44]:1
[7] linesearchProjection(f::Function, g::Function, h::Function, x0::Array{T,1} where T, direction::Function, step::Step, δ::Float64, stop::Function) in Main at In[44]:1
[8] linesearchProjection(f::Function, g::Function, h::Function, x0::Array{T,1} where T, direction::Function, step::Step, δ::Float64, stop::Function, nmax::Int64) in Main at In[44]:1

In [46]:
x, dfx, k = linesearchProjection(f, g, H, [1.0,1.0], steepestDirection, goldsteinstep)

([2.2844841247197536, 0.0], [-7.086547810786215e-7, 9.137936498879014], 42)

In [47]:
goldsteinstep.αmax = 2.0
x, dfx, k = linesearchProjection(f, g, H, [1.0,1.0], steepestDirection, goldsteinstep)

([2.2844841221111016, 0.0], [-8.198522607472114e-7, 9.137936488444407], 13)

In [48]:
goldsteinstep.αmax = 1000.0
x, dfx, k = linesearchProjection(f, g, H, [1.0,1.0], steepestDirection, goldsteinstep)

([2.284484129661535, 0.0], [-4.980043826208203e-7, 9.13793651864614], 23)

In [49]:
goldsteinstep.αmax = 200000.0
x, dfx, k = linesearchProjection(f, g, H, [1.0,1.0], steepestDirection, goldsteinstep)

([2.2844841268133513, 0.0], [-6.19412226399163e-7, 9.137936507253405], 9)

In [54]:
goldsteinstep.αmax = 0.0001
x, dfx, k = linesearchProjection(f, g, H, [1.0,1.0], steepestDirection, goldsteinstep)

([2.208818776646361, 0.0], [-3.0701250922439343, 8.835275106585444], 1000)

In [53]:
typeof(goldsteinstep)

LineSearchStep

Nous devrions être capable d'ajuster automatiquement $\alpha$max. Le schéma de région de confiance est particulièrement adapté pour cela.

### Troncature

Projeter n'est pas tronquer!

In [55]:
function (truncatedDirection::Bounds)(f::Function, g:: Function, h:: Function, x::Vector, αmax = 1.0)
    n = length(x)
    
    ∇f = g(x)

    α = αmax
    for i = 1:n
        if ∇f[i] > 0
            α = min(α,(x[i]-truncatedDirection.l[i])/∇f[i])
        elseif ∇f[i] < 0
            α = min(α,(x[i]-truncatedDirection.u[i])/∇f[i])
        end
    end
    
    d = -∇f*α

    return d, ∇f
end

In [56]:
function (truncatedDirection::Bounds)(x::Vector, d::Vector, αmax = 1.0)
    n = length(x)

    α = αmax
    for i = 1:n
        if d[i] < 0
            α = min(α,(truncatedDirection.l[i]-x[i])/d[i])
        elseif d[i] > 0
            α = min(α,(truncatedDirection.u[i]-x[i])/d[i])
        end
    end
    
    d *= α

    return d
end

In [57]:
x = [1,1]
d, dfx = steepestDirection(f,g,H,x)

([15.83879077652744, -22.16120922347256], [-15.83879077652744, 22.16120922347256])

In [58]:
d, dfx = projectedDirection(f,g,H,x)

([15.83879077652744, -1.0], [-15.83879077652744, 22.16120922347256])

In [59]:
x+d

2-element Array{Float64,1}:
 16.83879077652744
  0.0

In [60]:
truncatedDirection = Bounds(l, u)
d, dfx = truncatedDirection(f,g,H,x)

([0.7147078761275993, -1.0], [-15.83879077652744, 22.16120922347256])

In [61]:
x+d

2-element Array{Float64,1}:
 1.7147078761275993
 0.0

In [62]:
function linesearchTruncation(f::Function, g::Function, h:: Function,
        x0::Vector, direction::Function, steplength::Function,
        δ::Float64 = 1e-6, stop::Function = stopping, nmax::Int64 = 1000)
    
    k = 0
    x = x0
    δ2 = δ*δ
    n = length(x)

    # Compute the search direction
    d, ∇f = direction(f,g,h,x)
    s = truncatedDirection(x, d)
    println(s)

    while (stop(s, k, nmax, δ2) == false)
        # Compute the step length along d
        α = steplength(f,dfx,x,d)
        # Update the iterate
        x += α*s
        # Compute the search direction
        d, ∇f = direction(f,g,h,x)
        s = truncatedDirection(x, d)
        k += 1
    end

    return x, ∇f, k
end

linesearchTruncation (generic function with 4 methods)

In [64]:
x, dfx, k = linesearchTruncation(f, g, H, [1.0,1.0], steepestDirection, ArmijoStep)

[0.7147078761275993, -1.0]


([2.063773419593784, 7.048843440897225e-7], [-8.115694776214774, 8.255107776053283], 347)

In [66]:
f(x)

-28.57869819250214

La méthode s'est arrêtée avant de converger vers une solution, comme nous sommes limités à de très petits pas une fois proche de la frontière.

## Gradient projeté et région de confiance

Nous reproduisons d'abord l'algorithme de région de confiance de base.

In [67]:
struct BasicTrustRegion{T <: Real}
    η1:: T
    η2:: T
    γ1:: T
    γ2:: T
end

function BTRDefaults()
    return BasicTrustRegion(0.01,0.9,0.5,0.5)
end

struct UnconstrainedStep <: Step
end

mutable struct BTRState

    iter::Int
    x::Vector
    xcand::Vector
    g::Vector
    H::Matrix
    step::Vector
    Δ::Float64
    ρ::Float64

    tol::Float64

    trace
    keepTrace::Bool
    
    function BTRState()
        state = new()
        state.tol = 1e-6
        state.keepTrace = false
        return state
    end
end


function acceptCandidate!(state::BTRState, b::BasicTrustRegion)
    # If the iteration is successful, update the iterate
    if (state.ρ >= b.η1)
        return true
    else
        return false
    end
end

function updateRadius!(state::BTRState, b::BasicTrustRegion)
    if (state.ρ >= b.η2)
        stepnorm = norm(state.step)
        state.Δ = min(1e20,max(4*stepnorm,state.Δ))
    elseif (state.ρ >= b.η1)
        state.Δ *= b.γ2
    else
        state.Δ *= b.γ1
    end
end

function BFGSUpdate(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B - (Bs*Bs')/dot(s, Bs) + (y*y')/dot(s,y)
end

function stopCG(normg::Float64, normg0::Float64, k::Int, kmax::Int, χ::Float64 = 0.1, θ::Float64 = 0.5)
    if ((k == kmax) || (normg <= normg0*min(χ, normg0^θ)))
        return true
    else
        return false
    end
end

defaultState = BTRState()

BTRState(675390928, #undef, #undef, #undef, #undef, #undef, 0.0, 8.05683204e-316, 1.0e-6, #undef, false)

In [68]:
function btrStopUnconstrained(state::BTRState)
    nmax = 1000

    if ((dot(state.g,state.g) <= state.tol*state.tol) || (state.iter >= nmax))
        return true
    end
    
    return false
end

btrStopUnconstrained (generic function with 1 method)

In [69]:
function btrStopStep(state::BTRState)
    if (dot(state.step, state.step) < state.tol*state.tol)
        return true
    end
    
    return false
end

btrStopStep (generic function with 1 method)

In [70]:
function btr(f::Function, ∇f::Function, ∇2f::Function, Step::Function,
             x0::Vector, state:: BTRState = BTRState(), stop:: Function = btrStopUnconstrained,
             ApproxH::Bool = false, verbose::Bool = false)
    
    b = BTRDefaults()
    state.iter = 0
    state.x = x0
    n = length(x0)

    state.step = zeros(n)
    state.step[1] = +Inf   # prevent to step with a too small step

    tol2 = state.tol*state.tol
    
    state.g = zeros(n)
    
    fx = f(x0)
    state.g = ∇f(x0)
    state.Δ = 0.1*norm(state.g) # 1.0

    if (ApproxH)
        # A better initialization procedure should be used with quasi-Newton approximations
        # We could rely on some preconditioner.
        state.H = zeros(n,n)+I
        y = zeros(n)
        gcand = zeros(n)
    else
        state.H = ∇2f(x0)
    end
    
    nmax = 1000
    if (state.keepTrace)
        state.trace= x0'
    end
    
    function model(s:: AbstractVector{T}, g:: AbstractVector{T}, H:: AbstractMatrix{T}) where T
        return dot(s, g)+0.5*dot(s, H*s)
    end
    
    while (!stop(state))
        # Compute the step by approximately minimize the model
        state.step = Step(state)
        state.xcand = state.x+state.step
        
        # Compute the actual reduction over the predicted reduction
        fcand = f(state.xcand)
        state.ρ = (fcand-fx)/(model(state.step, state.g, state.H))

       if (ApproxH)
            gcand = ∇f(state.xcand)
            y = gcand-state.g
            sy = dot(state.step,y)
#            if (sy < 1e-6)
#                println(state.iter, ". ", state.ρ, " ", state.Δ, " ", norm(state.step), " ", (model(state.step, state.g, H)), " ", norm(y), " ", sy, " ", norm(state.g))
#            end
            state.H = ∇2f(H, y, state.step)
        end

        if (acceptCandidate!(state, b))
            state.x = copy(state.xcand)
            if (ApproxH == false)
                state.g = ∇f(state.x)
                state.H = ∇2f(state.x)
            else
                state.g = copy(gcand)
            end
            fx = fcand
        end

        if (state.keepTrace)
            state.trace= [state.trace ; state.x']
        end
        
        updateRadius!(state, b)
        state.iter += 1
    end
    
    return state
end

btr (generic function with 5 methods)

In [71]:
function CauchyStep(g::Vector, H::Matrix, Δ::Float64)
    q = dot(g,H*g)
    normg = norm(g)
    if (q <= 0)
        τ = 1.0
    else
        τ = min((normg*normg*normg)/(q*Δ),1.0)
    end
    return -τ*g*Δ/normg
end

function CauchyStep(state:: BTRState)
    return CauchyStep(state.g, state.H, state.Δ)
end

CauchyStep (generic function with 2 methods)

In [72]:
function TruncatedCG(g::Vector, H::Matrix, Δ::Float64)
    n = length(g)
    s = zeros(n)

    normg0 = norm(g)
    v = g
    d = -v
    gv = dot(g,v)
    norm2d = gv
    norm2s = 0
    sMd = 0
    k = 0
    Δ2 = Δ*Δ

    while (stopCG(norm(g), normg0, k, n) == false)
        Hd = H*d
        κ = dot(d,Hd)
 
        # Is the curvature negative in the direction d?
        if (κ <= 0)
            σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-dot(s,s))))/norm2d
            s += σ*d
            break
        end

        α = gv/κ

        # Check is the model minimizer is outside the trust region
        norm2s += α*(2*sMd+α*norm2d)
        if (norm2s >= Δ2)
            σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-dot(s,s))))/norm2d
            s += σ*d
            break
        end

        # The model minimizer is inside the trust region
        s += α*d
        g += α*Hd
        v = g
        newgv = dot(g,v)
        β = newgv/gv
        gv = newgv
        d = -v+β*d
        sMd = β*(sMd+α*norm2d)
        norm2d = gv+β*β*norm2d
        
        k += 1;
    end
    
    return s
end

function TruncatedCG(state:: BTRState)
    return TruncatedCG(state.g, state.H, state.Δ)
end

TruncatedCG (generic function with 2 methods)

In [73]:
state = btr(f, g, H, CauchyStep, [1.0,1.0])

BTRState(8, [2.306630130114104, -0.33230864878785144], [2.306630130114104, -0.33230864878785144], [1.063943741996809e-7, -1.2961098860841958e-10], [44.15289727064744 0.754635329776236; 0.754635329776236 34.761904044961156], [1.6874772340955503e-10, 1.3888848506182687e-7], 0.4460773308500046, 0.9748060332431441, 1.0e-6, #undef, false)

In [74]:
state = btr(f, g, H, TruncatedCG, [1.0,1.0])

BTRState(7, [2.3066301277034675, -0.3323086487317943], [2.3066301277034675, -0.3323086487317943], [8.526512829121202e-14, -1.0746958878371515e-13], [44.15289713679731 0.7546353369937435; 0.7546353369937435 34.76190399984316], [-3.547643308201812e-8, 6.156113901275451e-8], 0.8064365595032105, 1.0039546332276776, 1.0e-6, #undef, false)

Nous pouvons calculer le point de Cauchy généralisé en utilisant l'algorithme 12.2.2 du livre "Trust-Region Methods".

In [75]:
mutable struct BoundsConstrainedStep <: Step
    l:: Vector     # lower bounds
    u:: Vector     # Upper bounds
    β1:: Float64
    β2:: Float64
    μ:: Float64
    ν:: Float64
    
    function BoundsConstrainedStep(l::Vector, u::Vector,
                β1::Float64 = 0.1, β2::Float64 = 0.75, ν::Float64 = 0.2, μ::Float64 = 0.1)
        bcs = new()
        bcs.l = copy(l)
        bcs.u = copy(u)
        bcs.β1 = β1
        bcs.β2 = β2
        bcs.μ = μ
        return bcs
    end
end

In [76]:
GCP = BoundsConstrainedStep(l,u)

typeof(GCP) == BoundsConstrainedStep

true

In [79]:
function projectTangentConeBounds!(x::Vector, d::Vector, l::Vector, u::Vector, tol::Float64 = 1e-12)
    for i = 1:length(x)
        if (x[i]-l[i] < tol)
            d[i] = max(0, d[i])
        end
        if (u[i]-x[i] < tol)
            d[i] = min(0, d[i])
        end
    end
    
    return d
end

projectTangentConeBounds! (generic function with 2 methods)

In [80]:
function model(s::Vector, g::Vector, H::Matrix)
    return dot(s, g)+0.5*dot(s, H*s)
end

model (generic function with 1 method)

In [82]:
# Algorithm 12.2.2 in Conn, Gould, and Toint, "Trust-region methods", SIAM, 2000.

function (GCP::BoundsConstrainedStep)(x:: Vector, g::Vector, H::Matrix, Δ::Float64)
    tmin = 0
    tmax = Inf
    t = Δ/norm(g)
    Δ2 = Δ*Δ
    μ2 = GCP.μ^2
    s = zeros(length(x))   # prevents s to be a local variable of the while loop.
    
    cont = true
 
    # The condition on j is set as a crude safeguard.
    while (cont)
        p = boundsProjection(x-t*g, GCP.l, GCP.u)  # compute the point on the projected gradient path
        s = p-x
        mk = model(s, g, H)
        gs = dot(g,s)

        if ((dot(s,s) > Δ2) || (mk > GCP.β1*gs))
            tmax = t
        elseif ((dot(s,s) < (μ2*Δ2)) && (mk < GCP.β2*gs) &&
            (norm(projectTangentConeBounds!(p,-g,GCP.l,GCP.u)) > GCP.ν*abs(gs)/Δ))
            tmin = t
        else
            cont = false
            return s
        end
        
        if tmax < Inf
            t = (tmin+tmax)/2
        else
            t *= 2;
        end
    end
    
    return s
end

In [83]:
Δ = state.Δ = 1.1
x = state.x = [1, 0.9]
l = [0.0, 0.0]
u = [10.0, 10.0]
g(x), H(x), Δ

([-15.762204114225607, 20.48643987308266], [-10.537979187193045 -0.33353700157628285; -0.33353700157628285 16.866692361490067], 1.1)

In [213]:
s = GCP(x, g(x), H(x), Δ)

2-element Array{Float64,1}:
  0.670772847559197
 -0.8718163722810705

In [84]:
state.g = g(x)
state.H = H(x)

2×2 Array{Float64,2}:
 -10.538     -0.333537
  -0.333537  16.8667

In [88]:
function boundsstep(state:: BTRState)
    s = GCP(state.x, state.g, state.H, state.Δ)
#    println("Step = ", s, state.x, state.x+s)
    return s
end

boundsstep (generic function with 1 method)

In [89]:
s = boundsstep(state)

2-element Array{Float64,1}:
 3.9130355622418733e-7
 0.0

In [90]:
state = btr(f, g, H, boundsstep, [1.0,1.0], state, btrStopStep)

BTRState(23, [2.2844838949817237, 0.0], [2.2844838949817237, 0.0], [-1.0501561391151881e-5, 9.137935579926895], [42.62639999717041 4.0; 4.0 20.0], [-6.651174695448958e-7, 0.0], 1.3619714456525291, 0.9958346563098921, 1.0e-6, #undef, false)

In [221]:
state.x

2-element Array{Float64,1}:
 2.2844838949817237
 0.0

In progress...

Another possibility is to project the gradient, and when a constraint becomes active, to continue the search along the set of active constraints. See https://wiki.mcs.anl.gov/leyffer/images/0/01/07-bndCons.pdf

For the projected gradient, we can first determine the active set at the current iterate, incorporating information about the current search direction.

In [ ]:
function activeSetBounds(x:: Vector, direction:: Vector, b::Bounds, tol = 10*eps(Float64))
    n = length(x)
    # Initialize the number of free variables
    nfree = n  # nfree will contain the number of free variables
    # free designs which variables are free, and which variables are constrained to a bound.
    # free[i] = 1 if x[i] is free, and free[0] otherwise.
    free = ones(n)
    
    for i=1:n
        if (((abs(b.l[i]-x[i]) <= tol) && direction[i] < 0) || ((abs(x[i]-b.u[i]) <= tol) && direction[i] > 0))
            free[i] = 0
            nfree -= 1
        end
    end
    
    return free, nfree
end

In [ ]:
l = [0; 0]
u = [1; 1]
b = Bounds(l, u)
d = [-1; -1]
x = [0; 1]
free, nfree = activeSetBounds(x, d, b)

Knowing the active set, we can compute the restricted direction.

In [ ]:
function reducedDirectionBounds(d:: Vector, free:: Vector)
    return d.*free
end

In [ ]:
rd = reducedDirectionBounds(d, free)

We can use the reduced gradient to compute the Cauchy point. Recall the computation of the Cauchy step in the unconstrained case.

In [ ]:
function CauchyStep(g::Vector,H::Matrix,Δ::Float64)
    q = dot(g,H*g)
    normg = norm(g)

    if (q <= 0)
        τ = 1.0
    else
        τ = min((normg*normg*normg)/(q*Δ),1.0)
    end

    return -τ*g*Δ/normg
end

We will now examine the curvature with the reduced gradient instead of the gradient.

If the curvature is negative, we have to search along the projected gradient a step that does not lead outside the trust region.

If the curvature is positive, we will check if the model minimization is inside the intersection of the feasible set and the trust-region. If it is not, we again have to search along the projected gradient. In the context of conjugated gradient, if the trust-region boundary has not been reached yet, we update the set of free variables and computed the next search direction.

Projected conjugate gradient: https://epubs.siam.org/doi/pdf/10.1137/S1052623498345075

TO DO!